SAM2 Segmentation:

Used Segment Anything Model 2 (SAM2) to generate a mask of the white area in the image using the mask_generator.generate() function. The results were stored in the sam2_result variable.
Mask Inversion:

Since the goal was to calculate the length of the black area (the area not covered by the mask), the mask was inverted using inverted_mask = np.logical_not(mask). This effectively swapped the white and black areas in the mask.
Applying Closing Morphological Operation

To fill holes and smooth the contours of the inverted mask (the black area), a closing operation was performed using OpenCV's cv2.morphologyEx with cv2.MORPH_CLOSE. This is essential to ensure the skeletonization process yields a more accurate representation of the area's length.
Skeletonization:

The skeleton of the processed inverted mask was computed using skeleton = skeletonize(closed > 0) from the skimage.morphology module. The skeleton represents the essential, thin lines capturing the length of the area of interest.
Length Calculation:

The number of pixels belonging to the skeleton was counted using np.sum(skeleton). This pixel count was then converted to micrometers using a conversion factor specific to your image (you provided 400 / 300 in your original code).
Visualization:

The original image and the calculated skeleton with the length in micrometers were displayed side-by-side for easy comparison.
Steps Taken:

Access Mask from SAM2 Output: The segmentation mask (sam2_result[0]['segmentation']) was extracted from the sam2_result.
Invert the Mask: The np.logical_not() function was used to invert the mask, obtaining the black area's representation.
Apply Closing Operation: OpenCV function was used for processing the image before skeletonization. This was to remove holes inside the object and smooth contours for better skeletonization.
Skeletonize the Mask: skeletonize function from skimage.morphology was employed to obtain the skeleton.
Calculate Length: The number of pixels in the skeleton was converted into length in micrometers using a conversion factor.
Visualize Results: matplotlib.pyplot was used to display the results, showing both the original image and the calculated skeleton with length information.

In [ ]:
import numpy as np
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
import cv2 # Asegúrate de importar cv2

# Accede a la máscara
mask = sam2_result[0]['segmentation']

# Invierte la máscara
inverted_mask = np.logical_not(mask)

# Aplica la operación de cierre
kernel = np.ones((5,5),np.uint8)
closed = cv2.morphologyEx(inverted_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)


# Esqueletonización
skeleton = skeletonize(closed > 0)

# Cálculo de la longitud (reutilizando tu código)
skeleton_pixels = np.sum(skeleton)
pixels_to_microns = 400 / 300  # Factor de conversión
length_microns = skeleton_pixels * pixels_to_microns

# Mostrar resultados (reutilizando tu código)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image_bgr, cmap=plt.cm.gray) # Mostrando la imagen original en escala de grises
ax[0].axis('off')
ax[0].set_title('original', fontsize=20)

ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].axis('off')
ax[1].set_title(f'Length: {length_microns:.1f} µm', fontsize=20)

fig.tight_layout()
plt.show()